# Tutorial: Flash Unit Model with Modular Property Package

![](flash.svg)

## Learning Outcomes

- Demonstrate use of the flash unit model in IDAES


## Problem Statement

In this example, we will be flashing a liquid feed stream of benzene and toluene to form a liquid outlet stream and a vapor outlet stream. The inlet conditions are as follows:

**Inlet:**

Flow Rate = 100 mol/s

Mole Fraction (Benzene) = 0.6

Mole Fraction (Toluene) = 0.4

Temperature = 298 K

Pressure = 101325 Pa


For more details, please refer to the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable

## Importing necessary tools

In the following cell, we will be importing the necessary components from Pyomo and IDAES.

In [ ]:
# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, value

# Import the solver
from idaes.core.solvers import get_solver

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the flash unit model
from idaes.models.unit_models import Flash

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the modular property package to create a property block for the flowsheet
from idaes.models.properties.modular_properties.base.generic_property import GenericParameterBlock

# Import the BT_Ideal property package to create a configuration file for the GenericParameterBlock
from idaes.models.properties.modular_properties.examples.BT_ideal import configuration

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

## Setting up the flowsheet

In the following cell, we will create the `ConcreteModel` foundation, attach the steady state flowsheet, and declare the property parameter block that will used.

More information on this general workflow can be found here: https://idaes-pse.readthedocs.io/en/stable/how_to_guides/workflow/general.html

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False) # dynamic or ss flowsheet needs to be specified here

m.fs.properties = GenericParameterBlock(**configuration)

In the following cell, we will be creating the flash unit model, assigning a property package to it, and determining the initial degrees of freedom associated with the flash unit model.

In [ ]:
m.fs.flash = Flash(property_package=m.fs.properties)

DOF_initial = degrees_of_freedom(m)
print('The initial degrees of freedom are: {0}'.format(DOF_initial))

In [ ]:
assert DOF_initial == 7

### Fixing input specifications
In the following cell, we will be specifying the inlet conditions for the flash block and re-evaluating the degrees of freedom to ensure the problem is square (i.e. DOF=0). It will be assumed that the flash unit has a heat duty of 100,000 W and no pressure change.

In [ ]:
m.fs.flash.inlet.flow_mol.fix(100) # converting to mol/s as unit basis is mol/s
m.fs.flash.inlet.mole_frac_comp[0, "benzene"].fix(0.6)
m.fs.flash.inlet.mole_frac_comp[0, "toluene"].fix(0.4)
m.fs.flash.inlet.pressure.fix(101325) # Pa
m.fs.flash.inlet.temperature.fix(353) # K

m.fs.flash.heat_duty.fix(1e5) # W
m.fs.flash.deltaP.fix(0)

DOF_final = degrees_of_freedom(m)
print('The final degrees of freedom is: {0}'.format(DOF_final))

In [ ]:
assert DOF_final == 0

### Flowsheet Initialization

IDAES includes pre-written initialization routines for all unit models. Failing to initialize or having a poor intialization of a flowsheet may result in the problem being unsolvable. The output from initialization can be set to 7 different levels depending on the details required by the user. In general, when a particular output level is set, any information at that level and above gets picked up by logger. The default level taken by the logger is INFO.

More information on these levels can be found in the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable/reference_guides/logging.html

In [ ]:
m.fs.flash.initialize(outlvl=idaeslog.WARNING)

### Obtaining Simulation Results

In the following cell, the flowsheet will be solved using the IDAES `get_solver` tool. Setting `tee=True` will display the solver output.

In [ ]:
solver = get_solver()
result = solver.solve(m, tee=True)

In [ ]:
from pyomo.opt import TerminationCondition, SolverStatus

# Check if termination condition is optimal
assert result.solver.termination_condition == TerminationCondition.optimal
assert result.solver.status == SolverStatus.ok

### View Results
As expected, the report below shows that part of the feed stream has been vaporized such that there is a pure liquid outlet stream and a pure vapor outlet stream

In [ ]:
m.fs.flash.report()

In [ ]:
import pytest

# # Check liquid outlet results
assert value(m.fs.flash.liq_outlet.flow_mol[0]) == pytest.approx(97.559, rel=1e-3)
assert value(m.fs.flash.liq_outlet.mole_frac_comp[0, "benzene"]) == pytest.approx(0.59531, rel=1e-3)
assert value(m.fs.flash.liq_outlet.mole_frac_comp[0, "toluene"]) == pytest.approx(0.40469, rel=1e-3)
assert value(m.fs.flash.liq_outlet.pressure[0]) == pytest.approx(101325, rel=1e-3)
assert value(m.fs.flash.liq_outlet.temperature[0]) == pytest.approx(362.69, rel=1e-3)


# Check vapor outlet results
assert value(m.fs.flash.vap_outlet.flow_mol[0]) == pytest.approx(2.4408, rel=1e-3)
assert value(m.fs.flash.vap_outlet.mole_frac_comp[0, "benzene"]) == pytest.approx(0.7873, rel=1e-3)
assert value(m.fs.flash.vap_outlet.mole_frac_comp[0, "toluene"]) == pytest.approx(0.2127, rel=1e-3)
assert value(m.fs.flash.vap_outlet.pressure[0]) == pytest.approx(101325, rel=1e-3)
assert value(m.fs.flash.vap_outlet.temperature[0]) == pytest.approx(362.69, rel=1e-3)
